### <a name="service">Service definition

In [26]:
service = dict([('title', 'SATCEN-02-02-01 - Detection of potential illegal mining activities'),
                ('abstract', 'SATCEN-02-02-01 - Detection of potential illegal mining activities'),
                ('id', 'ewf-satcen-02-02-01')])

### Service parameters

In [27]:
aoi = dict([('id', 'aoi'),
            ('title', 'Area of interest (bbox)'),
            ('abstract', 'Area of interest defined as a bounding box'),
            ('value', '-70.5659,-13.0922,-69.1411,-12.4567')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the Sentinel-1 stack of products' identifiers

In [28]:
input_identifiers = ( 'S2B_MSIL2A_20190308T144729_N0211_R139_T19LDG_20190308T185300',
       'S2B_MSIL2A_20190308T144729_N0211_R139_T19LDF_20190308T185300', 'S2B_MSIL2A_20190206T144729_N0211_R139_T19LCF_20190206T185013',
                    'S2B_MSIL2A_20190206T144729_N0211_R139_T19LDF_20190206T185013') 

**Input references**

This is the Sentinel-1 stack catalogue references

In [29]:
input_references = ( 'https://catalog.terradue.com/sentinel2/search?format=atom&uid=S2B_MSIL2A_20190308T144729_N0211_R139_T19LDG_20190308T185300',
       'https://catalog.terradue.com/sentinel2/search?format=atom&uid=S2B_MSIL2A_20190308T144729_N0211_R139_T19LDF_20190308T185300',
                   'https://catalog.terradue.com/sentinel2/search?format=atom&uid=S2B_MSIL2A_20190206T144729_N0211_R139_T19LCF_20190206T185013',
                   'https://catalog.terradue.com/sentinel2/search?format=atom&uid=S2B_MSIL2A_20190206T144729_N0211_R139_T19LDF_20190206T185013') 

**Data path**

This path defines where the data is staged-in. 

In [30]:
data_path = "/workspace/data2"

### <a name="workflow">Workflow

In [31]:
from shapely.geometry import box
import numpy as np
import cioppy
import geopandas as gpd
from shapely.wkt import loads
import pandas as pd
import math
import os
import datetime
import lxml.etree as etree
import ogr
from xml.etree import ElementTree
from functools import partial
import pyproj
from shapely.ops import transform
import gdal
import osr
import sys
sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())

sys.path.append('/opt/OTB/lib/python')
sys.path.append('/opt/OTB/lib/libfftw3.so.3')
os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB/lib/otb/applications'
os.environ['LD_LIBRARY_PATH'] = '/opt/OTB/lib'
os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB/lib/otb/applications'

import otbApplication

In [32]:
ciop = cioppy.Cioppy()

In [33]:
temp_searches = []

for index, reference in enumerate(input_references):
    
    search_temp = gpd.GeoDataFrame(ciop.search(end_point=reference,
                                          params=[],
                                           output_fields='self,track,enclosure,identifier,wkt,startdate,enddate,platform,cc', 
                                           model='EOP'))
    
    
    search_temp['local_path'] = os.path.join(data_path, input_identifiers[index])
    
    temp_searches.append(search_temp)
    
search = gpd.GeoDataFrame( pd.concat(temp_searches, ignore_index=True)) 

search['geometry'] = search['wkt'].apply(loads)
search['cc'] = pd.to_numeric(search['cc'])
search['startdate'] = pd.to_datetime(search['startdate'])
search['enddate'] = pd.to_datetime(search['enddate'])
    
    

In [34]:
search

,cc,enclosure,enddate,identifier,platform,self,startdate,track,wkt,local_path,geometry
0,5.084996,https://store.terradue.com/download/sentinel2/...,2019-03-08 14:47:29.024,S2B_MSIL2A_20190308T144729_N0211_R139_T19LDG_2...,S2B,https://catalog.terradue.com/sentinel2/search?...,2019-03-08 14:47:29.024,139,"POLYGON((-68.910095 -12.7524314685901,-68.9104...",/workspace/data2/S2B_MSIL2A_20190308T144729_N0...,"POLYGON ((-68.910095 -12.7524314685901, -68.91..."
1,45.163561,https://store.terradue.com/download/sentinel2/...,2019-03-08 14:47:29.024,S2B_MSIL2A_20190308T144729_N0211_R139_T19LDF_2...,S2B,https://catalog.terradue.com/sentinel2/search?...,2019-03-08 14:47:29.024,139,"POLYGON((-68.90976 -13.656868384074,-68.910126...",/workspace/data2/S2B_MSIL2A_20190308T144729_N0...,"POLYGON ((-68.90976000000001 -13.656868384074,..."
2,48.573114,https://store.terradue.com/download/sentinel2/...,2019-02-06 14:47:29.024,S2B_MSIL2A_20190206T144729_N0211_R139_T19LCF_2...,S2B,https://catalog.terradue.com/sentinel2/search?...,2019-02-06 14:47:29.024,139,"POLYGON((-70.12924 -13.6538874254132,-70.11722...",/workspace/data2/S2B_MSIL2A_20190206T144729_N0...,"POLYGON ((-70.12924 -13.6538874254132, -70.117..."
3,31.852275,https://store.terradue.com/download/sentinel2/...,2019-02-06 14:47:29.024,S2B_MSIL2A_20190206T144729_N0211_R139_T19LDF_2...,S2B,https://catalog.terradue.com/sentinel2/search?...,2019-02-06 14:47:29.024,139,"POLYGON((-69.9216 -12.7519041760794,-69.91202 ...",/workspace/data2/S2B_MSIL2A_20190206T144729_N0...,"POLYGON ((-69.9216 -12.7519041760794, -69.9120..."


In [35]:
def analyse(row):
    
    series = dict()
   
    series['utm_zone'] = row['identifier'][39:41]
    series['latitude_band'] = row['identifier'][41]
    series['grid_square']  = row['identifier'][42:44]

    
    return pd.Series(series)

In [36]:
search = search.merge(search.apply(lambda row: analyse(row ), axis=1),
                                    left_index=True,
                                  right_index=True)

In [37]:
search

,cc,enclosure,enddate,identifier,platform,self,startdate,track,wkt,local_path,geometry,grid_square,latitude_band,utm_zone
0,5.084996,https://store.terradue.com/download/sentinel2/...,2019-03-08 14:47:29.024,S2B_MSIL2A_20190308T144729_N0211_R139_T19LDG_2...,S2B,https://catalog.terradue.com/sentinel2/search?...,2019-03-08 14:47:29.024,139,"POLYGON((-68.910095 -12.7524314685901,-68.9104...",/workspace/data2/S2B_MSIL2A_20190308T144729_N0...,"POLYGON ((-68.910095 -12.7524314685901, -68.91...",DG,L,19
1,45.163561,https://store.terradue.com/download/sentinel2/...,2019-03-08 14:47:29.024,S2B_MSIL2A_20190308T144729_N0211_R139_T19LDF_2...,S2B,https://catalog.terradue.com/sentinel2/search?...,2019-03-08 14:47:29.024,139,"POLYGON((-68.90976 -13.656868384074,-68.910126...",/workspace/data2/S2B_MSIL2A_20190308T144729_N0...,"POLYGON ((-68.90976000000001 -13.656868384074,...",DF,L,19
2,48.573114,https://store.terradue.com/download/sentinel2/...,2019-02-06 14:47:29.024,S2B_MSIL2A_20190206T144729_N0211_R139_T19LCF_2...,S2B,https://catalog.terradue.com/sentinel2/search?...,2019-02-06 14:47:29.024,139,"POLYGON((-70.12924 -13.6538874254132,-70.11722...",/workspace/data2/S2B_MSIL2A_20190206T144729_N0...,"POLYGON ((-70.12924 -13.6538874254132, -70.117...",CF,L,19
3,31.852275,https://store.terradue.com/download/sentinel2/...,2019-02-06 14:47:29.024,S2B_MSIL2A_20190206T144729_N0211_R139_T19LDF_2...,S2B,https://catalog.terradue.com/sentinel2/search?...,2019-02-06 14:47:29.024,139,"POLYGON((-69.9216 -12.7519041760794,-69.91202 ...",/workspace/data2/S2B_MSIL2A_20190206T144729_N0...,"POLYGON ((-69.9216 -12.7519041760794, -69.9120...",DF,L,19


In [38]:


aoi_wkt = box(*[float(i) for i in aoi['value'].split(',')]).wkt

min_lon, min_lat,  max_lon, max_lat = [float(i) for i in aoi['value'].split(',')]


In [39]:
def get_band_path(row, band):
    
    ns = {'xfdu': 'urn:ccsds:schema:xfdu:1',
          'safe': 'http://www.esa.int/safe/sentinel/1.1',
          'gml': 'http://www.opengis.net/gml'}
    
    path_manifest = os.path.join(row['local_path'],
                                 row['identifier'] + '.SAFE', 
                                'manifest.safe')
    
    root = etree.parse(path_manifest)
    
    bands = [band]

    for index, band in enumerate(bands):

        sub_path = os.path.join(row['local_path'],
                                row['identifier'] + '.SAFE',
                                root.xpath('//dataObjectSection/dataObject/byteStream/fileLocation[contains(@href,("%s%s")) and contains(@href,("%s")) ]' % (row['latitude_band'],
                                row['grid_square'], 
                                band), 
                                  namespaces=ns)[0].attrib['href'][2:])
    
    return sub_path

In [40]:
bands = ['B02', 'B04', 'B11', 'B12']

In [41]:
for index, band in enumerate(bands):
    
    vrt_bands = []
    
    for j, row in search.iterrows():
        
        vrt_bands.append(get_band_path(row, band))
    
    
    ds = gdal.BuildVRT('{0}.vrt'.format(band),
                       vrt_bands,
                       srcNodata=0,
                       xRes=10, 
                       yRes=10)
    ds.FlushCache()
    
    
    gdal.Translate(band + '.tif', 
                   ds, 
                   outputType=gdal.GDT_Float32, 
                   projWin=[min_lon, max_lat, max_lon, min_lat],
                   projWinSRS='EPSG:4326')

In [42]:
band_expressions = ['im4b1 !=0 ? im3b1 / im4b1 : 0', 
                    'im1b1 !=0 ? im2b1 / im1b1 : 0', 
                    'im3b1 !=0 ? im2b1 / im3b1 : 0']

In [43]:
BandMathX = otbApplication.Registry.CreateApplication("BandMathX")

BandMathX.SetParameterStringList('il', [b + '.tif' for b in bands])

BandMathX.SetParameterString('out', 'temp.tif')

BandMathX.SetParameterString('exp', ';'.join(band_expressions))

BandMathX.ExecuteAndWriteOutput()

0

In [46]:


output_startdate = min(search['startdate'])
output_stopdate = max(search['enddate'])

In [48]:
output_stopdate

Timestamp('2019-03-08 14:47:29.024000')

In [49]:
date_format = '%Y%m%dT%H%m%S'

output_name = 'MINERAL-COMPOSITE-{0}-{1}'.format(output_startdate.strftime(date_format), 
                                                 output_stopdate.strftime(date_format))

In [50]:
Convert = otbApplication.Registry.CreateApplication("Convert")


Convert.SetParameterString("in", 'temp.tif')

Convert.SetParameterString("out", output_name)

Convert.SetParameterString("type","linear")

Convert.SetParameterString("channels","rgb")

Convert.ExecuteAndWriteOutput()

0

### Clean-up

In [51]:
for band in bands:
    for extension in ['.tif', '.vrt']:
        os.remove(band + extension)
        
os.remove('temp.tif')

### Metadata about results


In [52]:
def get_image_wkt(product):
    
    src = gdal.Open(product)
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()

    max_x = ulx + (src.RasterXSize * xres)
    min_y = uly + (src.RasterYSize * yres)
    min_x = ulx 
    max_y = uly

    source = osr.SpatialReference()
    source.ImportFromWkt(src.GetProjection())

    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)

    transform = osr.CoordinateTransformation(source, target)

    result_wkt = box(transform.TransformPoint(min_x, min_y)[0],
                     transform.TransformPoint(min_x, min_y)[1],
                     transform.TransformPoint(max_x, max_y)[0],
                     transform.TransformPoint(max_x, max_y)[1]).wkt
    
    return result_wkt

In [53]:
for properties_file in ['result', 'stage-in', output_name]:

    date_format = '%Y-%m-%dT%H:%m:%SZ'
    
    if properties_file == 'result':
        
        title = 'Reproducibility notebook used for generating {0}'.format(output_name)
   
    elif properties_file == 'stage-in':

        title = 'Reproducibility stage-in notebook for Sentinel-2 data for generating {0}'.format(output_name)
    
    else: 
      
        title = 'Mineral alteration index from {0} to {1}'.format(output_startdate.strftime(date_format),
                                                                  output_stopdate.strftime(date_format))
        
    with open(properties_file + '.properties', 'wb') as file:
        file.write('title={0}\n'.format(title))
        file.write('date={0}/{1}\n'.format(output_startdate.strftime(date_format),
                                           output_stopdate.strftime(date_format)))
        file.write('geometry={0}'.format(get_image_wkt(output_name + '.tif')))


### License




This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.